# Reconocimiento de Entidades Nombradas (NER)


### Autores: Lucas Tarazona y Samuel Arteaga

### 1. Importación de librerías

In [2]:
from transformers import pipeline
import spacy
from sklearn.metrics import classification_report

In [3]:
textos = [
    "El contrato entre Empresa XYZ y el proveedor ABC establece que la entrega de los bienes deberá realizarse en un plazo no mayor a 30 días desde la firma del acuerdo.",
    "Según la Ley 25/2018 de Protección de Datos, todas las empresas deben garantizar la privacidad de la información de sus clientes y empleados."
]

In [5]:
nlp = spacy.load("es_core_news_sm")  # Modelo en español

for texto in textos:
    doc = nlp(texto)
    print("\nTexto:", texto)
    print("Entidades reconocidas:")
    for ent in doc.ents:
        print(f" - {ent.text} ({ent.label_})")


Texto: El contrato entre Empresa XYZ y el proveedor ABC establece que la entrega de los bienes deberá realizarse en un plazo no mayor a 30 días desde la firma del acuerdo.
Entidades reconocidas:
 - Empresa (LOC)
 - ABC (ORG)

Texto: Según la Ley 25/2018 de Protección de Datos, todas las empresas deben garantizar la privacidad de la información de sus clientes y empleados.
Entidades reconocidas:
 - Según la Ley (PER)
 - Protección de Datos (LOC)


In [6]:
ner_pipeline = pipeline("ner", model="PlanTL-GOB-ES/roberta-base-bne", aggregation_strategy="simple")

for texto in textos:
    print("\nTexto:", texto)
    entidades = ner_pipeline(texto)
    for ent in entidades:
        print(f" - {ent['word']} ({ent['entity_group']})")

c:\Users\ESP\Desktop\ModelosIA\.venv\Lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\ESP\.cache\huggingface\hub\models--PlanTL-GOB-ES--roberta-base-bne. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Some weights of RobertaForTokenClassification were not initialized from the model checkpoint at PlanTL-


Texto: El contrato entre Empresa XYZ y el proveedor ABC establece que la entrega de los bienes deberá realizarse en un plazo no mayor a 30 días desde la firma del acuerdo.
 - El contrato (LABEL_1)
 -  entre (LABEL_0)
 -  Empresa XYZ y el proveedor ABC establece (LABEL_1)
 -  que (LABEL_0)
 -  la (LABEL_1)
 -  entrega de (LABEL_0)
 -  los bienes (LABEL_1)
 -  deberá realizarse en (LABEL_0)
 -  un plazo no mayor a 30 días desde (LABEL_1)
 -  la (LABEL_0)
 -  firma (LABEL_1)
 -  del (LABEL_0)
 -  acuerdo. (LABEL_1)

Texto: Según la Ley 25/2018 de Protección de Datos, todas las empresas deben garantizar la privacidad de la información de sus clientes y empleados.
 - Según (LABEL_1)
 -  la (LABEL_0)
 -  Ley (LABEL_1)
 -  25/ (LABEL_0)
 - 2018 (LABEL_1)
 -  de (LABEL_0)
 -  Protección (LABEL_1)
 -  de (LABEL_0)
 -  Datos, todas las empresas (LABEL_1)
 -  deben (LABEL_0)
 -  garantizar la privacidad (LABEL_1)
 -  de (LABEL_0)
 -  la información (LABEL_1)
 -  de (LABEL_0)
 -  sus clientes y e

In [7]:
# Supongamos que tenemos etiquetas reales y predicciones (esto es solo un ejemplo)
y_true = ["ORG", "LAW"]  # Empresa XYZ → ORG, Ley 25/2018 → LAW
y_pred = ["ORG", "LAW"]  # Modelo predice correctamente

print(classification_report(y_true, y_pred))

              precision    recall  f1-score   support

         LAW       1.00      1.00      1.00         1
         ORG       1.00      1.00      1.00         1

    accuracy                           1.00         2
   macro avg       1.00      1.00      1.00         2
weighted avg       1.00      1.00      1.00         2

